In [ ]:
!pip install mediapipe
!pip install mediapipe-model-maker

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Conv2DTranspose, UpSampling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, ImageDataGenerator
from google.colab import drive

drive.mount('/content/drive')

train_folder = '/content/drive/MyDrive/face_mesh_dataset/Training'
test_folder = '/content/drive/MyDrive/face_mesh_dataset/Testing'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_folder,
    target_size=(96, 96),  # Adjusted target size
    batch_size=32,
    class_mode='sparse'
)

test_generator = datagen.flow_from_directory(
    test_folder,
    target_size=(96, 96),  # Adjusted target size
    batch_size=32,
    class_mode='sparse'
)
model = Sequential()
model.add(Conv2D(32, (1, 1), activation='relu', input_shape=(96, 96, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling2D((2, 2)))


model.add(Conv2DTranspose(32,(3,3)))


model.add(Conv2D(64, (1, 1), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling2D((2,2)))




model.add(Conv2D(128, (1, 1), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling2D((2,2)))




model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history=model.fit(train_generator, epochs=50, validation_data=test_generator)

Found 4211 images belonging to 4 classes.
Found 1058 images belonging to 4 classes.
Epoch 1/50
132/132 [==============================] - 16s 91ms/step - loss: 1.2491 - accuracy: 0.3745 - val_loss: 1.1633 - val_accuracy: 0.4622
Epoch 2/50
132/132 [==============================] - 11s 84ms/step - loss: 1.1282 - accuracy: 0.4614 - val_loss: 1.1043 - val_accuracy: 0.5132
Epoch 3/50
132/132 [==============================] - 11s 84ms/step - loss: 1.0456 - accuracy: 0.5039 - val_loss: 1.0214 - val_accuracy: 0.5671
Epoch 4/50
132/132 [==============================] - 12s 91ms/step - loss: 0.9843 - accuracy: 0.5512 - val_loss: 0.9693 - val_accuracy: 0.5775
Epoch 5/50
132/132 [==============================] - 11s 81ms/step - loss: 0.9267 - accuracy: 0.5747 - val_loss: 0.9629 - val_accuracy: 0.5851
Epoch 6/50
132/132 [==============================] - 12s 88ms/step - loss: 0.8554 - accuracy: 0.6143 - val_loss: 0.9296 - val_accuracy: 0.5964
Epoch 7/50
132/132 [==============================] 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()